# Creating our own AlexNet from pre-built weights!

In [17]:
#### Weights from Michael Guerzhoy and Davi Frossard
# http://www.cs.toronto.edu/~guerzhoy/tf_alexnet/

![AlexNet](images/alexnet_architecture.png)

In [18]:
# Python > 3, TensorFlow > 1.0
import tensorflow as tf
import numpy as np

# Import pretrained Variables from .npy file

In [19]:
variable_data = np.load("saved_models/bvlc_alexnet.npy", encoding='bytes').item()
#variable_data2 = np.load("bvlc_alexnet.npy", encoding='bytes').item()

try:
    if variable_data2:
        old_idx = ['conv3/W', 'conv4/W', 'fc6/b', 'conv2/W', 'fc7/W', 'conv5/b', 'fc8/b', 'conv1/W', 'fc6/W', 'conv2/b', 'conv4/b', 'fc8/W', 'conv1/b', 'conv5/W', 'fc7/b', 'conv3/b']
        new_idx = ['conv3', 'conv4', 'fc6', 'conv2', 'fc7', 'conv5', 'fc8', 'conv1', 'fc6', 'conv2', 'conv4', 'fc8', 'conv1', 'conv5', 'fc7', 'conv3']
        variable_data = {}
        for nidx in  new_idx:
            variable_data[nidx] = [None,None]
            variable_data[nidx][0] = variable_data2[nidx+'/W'][1]
            variable_data[nidx][1] = variable_data2[nidx+'/b'] 
            print (variable_data[nidx][0].shape,variable_data[nidx][1].shape)
    print (type(variable_data2))
except:
    print ("using original data")

using original data


In [20]:
print (type(variable_data))
print (variable_data.keys())



<class 'dict'>
dict_keys(['conv4', 'conv2', 'conv5', 'fc7', 'conv1', 'conv3', 'fc6', 'fc8'])


In [21]:
conv1_preW = variable_data["conv1"][0]
conv1_preb = variable_data["conv1"][1]
print(conv1_preW.shape)
print(conv1_preb.shape)

(11, 11, 3, 96)
(96,)


In [22]:
conv2_preW = variable_data["conv2"][0]
conv2_preb = variable_data["conv2"][1]
print(conv2_preW.shape)
print(conv2_preb.shape)

(5, 5, 48, 256)
(256,)


In [23]:
conv3_preW = variable_data["conv3"][0]
conv3_preb = variable_data["conv3"][1]
print(conv3_preW.shape)
print(conv3_preb.shape)

(3, 3, 256, 384)
(384,)


In [24]:
conv4_preW = variable_data["conv4"][0]
conv4_preb = variable_data["conv4"][1]
print(conv4_preW.shape)
print(conv4_preb.shape)

(3, 3, 192, 384)
(384,)


In [25]:
conv5_preW = variable_data["conv5"][0]
conv5_preb = variable_data["conv5"][1]
print(conv5_preW.shape)
print(conv5_preb.shape)

(3, 3, 192, 256)
(256,)


In [26]:
fc6_preW = variable_data["fc6"][0]
fc6_preb = variable_data["fc6"][1]
print(fc6_preW.shape)
print(fc6_preb.shape)

(9216, 4096)
(4096,)


In [27]:
fc7_preW = variable_data["fc7"][0]
fc7_preb = variable_data["fc7"][1]
print(fc7_preW.shape)
print(fc7_preb.shape)

(4096, 4096)
(4096,)


In [28]:
fc8_preW = variable_data["fc8"][0]
fc8_preb = variable_data["fc8"][1]
print(fc8_preW.shape)
print(fc8_preb.shape)

(4096, 1000)
(1000,)


In [29]:
import cat_dog_queue

# Create the huge AlexNet!

In [30]:
pixel_depth = 255.0
resized_height = 227
resized_width = 227
num_channels = 3

graph = tf.Graph()

with graph.as_default():
    x = tf.placeholder(tf.uint8, [None, None, None, num_channels],
                       name='input')
    
    to_float = tf.cast(x, tf.float32)
    resized = tf.image.resize_images(to_float, (resized_height, resized_width))
    
    # Convolution 1
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(conv1_preW, name='weights')
        biases = tf.Variable(conv1_preb, name='biases')
        conv = tf.nn.conv2d(resized, kernel, [1, 4, 4, 1], padding="SAME")
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope)

    # Local response normalization 2
    radius = 2
    alpha = 2e-05
    beta = 0.75
    bias = 1.0
    lrn1 = tf.nn.local_response_normalization(conv1,
                                              depth_radius=radius,
                                              alpha=alpha,
                                              beta=beta,
                                              bias=bias)

    # Maxpool 1
    pool1 = tf.nn.max_pool(lrn1,
                           ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID',
                           name='pool1')

    # Convolution 2
    with tf.name_scope('conv2') as scope:

        kernel = tf.Variable(conv2_preW, name='weights')
        biases = tf.Variable(conv2_preb, name='biases')

        input_a, input_b = tf.split(axis=3,
                                    num_or_size_splits=2,
                                    value=pool1)
        kernel_a, kernel_b = tf.split(axis=3,
                                      num_or_size_splits=2,
                                      value=kernel)

        with tf.name_scope('A'):
            conv_a = tf.nn.conv2d(input_a, kernel_a, [1, 1, 1, 1], padding="SAME")        

        with tf.name_scope('B'):
            conv_b = tf.nn.conv2d(input_b, kernel_b, [1, 1, 1, 1], padding="SAME")

        conv = tf.concat(axis=3, values=[conv_a, conv_b])
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope)

    # Local response normalization 2
    radius = 2
    alpha = 2e-05
    beta = 0.75
    bias = 1.0
    lrn2 = tf.nn.local_response_normalization(conv2,
                                              depth_radius=radius,
                                              alpha=alpha,
                                              beta=beta,
                                              bias=bias)

    # Maxpool 2
    pool2 = tf.nn.max_pool(lrn2,
                           ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID',
                           name='pool2')

    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(conv3_preW, name='weights')
        biases = tf.Variable(conv3_preb, name='biases')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding="SAME")
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope)


    with tf.name_scope('conv4') as scope:

        kernel = tf.Variable(conv4_preW, name='weights')
        biases = tf.Variable(conv4_preb, name='biases')

        input_a, input_b = tf.split(axis=3,
                                    num_or_size_splits=2,
                                    value=conv3)
        kernel_a, kernel_b = tf.split(axis=3,
                                      num_or_size_splits=2,
                                      value=kernel)

        with tf.name_scope('A'):
            conv_a = tf.nn.conv2d(input_a, kernel_a, [1, 1, 1, 1], padding="SAME")        

        with tf.name_scope('B'):
            conv_b = tf.nn.conv2d(input_b, kernel_b, [1, 1, 1, 1], padding="SAME")

        conv = tf.concat(axis=3, values=[conv_a, conv_b])
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name=scope)


    with tf.name_scope('conv5') as scope:

        kernel = tf.Variable(conv5_preW, name='weights')
        biases = tf.Variable(conv5_preb, name='biases')

        input_a, input_b = tf.split(axis=3,
                                    num_or_size_splits=2,
                                    value=conv4)
        kernel_a, kernel_b = tf.split(axis=3,
                                      num_or_size_splits=2,
                                      value=kernel)

        with tf.name_scope('A'):
            conv_a = tf.nn.conv2d(input_a, kernel_a, [1, 1, 1, 1], padding="SAME")        

        with tf.name_scope('B'):
            conv_b = tf.nn.conv2d(input_b, kernel_b, [1, 1, 1, 1], padding="SAME")

        conv = tf.concat(axis=3, values=[conv_a, conv_b])
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope)


    # Maxpool 2
    pool5 = tf.nn.max_pool(conv5,
                           ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID',
                           name='pool5')

    # Fully connected 6
    with tf.name_scope('fc6'):
        weights = tf.Variable(fc6_preW, name='fc6_weights')
        bias = tf.Variable(fc6_preb, name='fc6_bias')
        shape = tf.shape(pool5)
        size = shape[1] * shape[2] * shape[3]
        fc6 = tf.nn.relu_layer(tf.reshape(pool5, [-1, size]),
                               weights, bias, name='relu')

    # Fully connected 7
    with tf.name_scope('fc7'):
        weights = tf.Variable(fc7_preW, name='weights')
        bias = tf.Variable(fc7_preb, name='bias')
        fc7 = tf.nn.relu_layer(fc6, weights, bias, name='relu')

    # Fully connected 8
    with tf.name_scope('fc8'):
        weights = tf.Variable(fc8_preW, name='weights')
        bias = tf.Variable(fc8_preb, name='bias')
        # fc8 = tf.matmul(fc7, weights) + bias
        fc8 = tf.nn.xw_plus_b(fc7, weights, bias)

    softmax = tf.nn.softmax(fc8)

    init = tf.global_variables_initializer()


In [31]:
sess = tf.Session(graph=graph)
sess.run(init)

writer = tf.summary.FileWriter('tensorboard/alexnet', graph=graph)
writer.close()

# Exporting the Entire Model

[Check out the official tutorial for more details](https://www.tensorflow.org/versions/master/how_tos/meta_graph/index.html)

In [32]:
with graph.as_default():
    saver = tf.train.Saver()
    save_path = saver.save(sess, 'saved_models/alex_vars')